In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import random

from sklearn.cluster import KMeans

## Helper functions

In [ ]:
# flattens list of points [(x,y), (x,y), ...] into [x,y,x,y,x,y,...]
def flatten(points):
  return [v for p in points for v in p]

# turn [x,y,x,y,x,y,...] into [(x,y), (x,y), ...]
def unflatten(list_of_vals):
  # get separate lists for x and y values
  xs = list_of_vals[0::2] # go through list, starting at 0, skipping by 2
  ys = list_of_vals[1::2] # go through list, starting at 1, skipping by 2

  # put the pairs into list of pairs
  return [[x,y] for x,y in zip(xs,ys)]

In [ ]:
# find closest point to p in list of points ps
def closest_point(p, ps):

  # euclidean dist to p (without sqrt)
  def dist_to_p(op):
    return (p[0] - op[0]) ** 2 + (p[1] - op[1]) ** 2

  # sort by distance to p
  sorted_by_dist_to_p = sorted(ps, key=dist_to_p)

  # first one is the closest
  return sorted_by_dist_to_p[0]

## Input data

In [ ]:
# Creating 4 lists of random points as stand-in for contours.
# Each list has a different number of values between -2 and 12.

random_lists = []
for lcnt in range(4):
  list_len = random.randrange(16, 24, 2)
  list_min = random.uniform(-2, 2)
  list_max = random.uniform(8, 12)
  random_lists.append([random.uniform(list_min, list_max) for cnt in range(list_len)])

# list lengths
[len(l) for l in random_lists]

## KMeans to make lengths consistent

In [ ]:
min_len = min([len(l) for l in random_lists])
min_points = min_len // 2

random_lists_same_len = []

# for each list/contour
for l in random_lists:
  
  points = unflatten(l)

  # cluster into min_points clusters
  km = KMeans(n_clusters=min_points)

  # x and y are columns and each point is a sample
  clusters = km.fit_transform(points)

  # if your points are dense, you can probably use the cluster centers as the new points
  random_lists_same_len.append(km.cluster_centers_.tolist())

  # the more correct way to do this is to look in the original list for the closest point to each cluster center
  random_lists_same_len.append([
    closest_point(p, points) for p in km.cluster_centers_
  ])

In [ ]:
# create names for columns
col_names = flatten([[f"x{i}", f"y{i}"] for i in range(min_points)])

# flatten each list of points
df_vals = [flatten(points) for points in random_lists_same_len]

In [ ]:
# DataFrame it
df = pd.DataFrame(df_vals, columns=col_names)

# repeated rows due to appending random_lists_same_len twice in each iteration in the cell above
df

## Center points

In [ ]:
# given a row, center it's data points on 0,0
# NOTE: doesn't work for very unsymmetrical silhouettes
def center_row_flat(row):
  vmin,vmax = row.min(), row.max()
  vrange = (vmax - vmin)

  # scale row to [0,1]
  row_01 = (row - vmin) / vrange

  # scale row to [-vrange/2, vrange/2]
  row_c = vrange * (row_01 - 0.5)
  return row_c

# given a row, center it's data points on 0,0
def center_row(row):
  # use 2D points, so we center x and y
  row_p = np.array(unflatten(row))

  vmin,vmax = row_p.min(axis=0), row_p.max(axis=0)
  vrange = (vmax - vmin)

  # scale row to [0,1]
  row_01 = (row_p - vmin) / vrange

  # scale row to [-vrange/2, vrange/2]
  row_c = vrange * (row_01 - 0.5)
  return pd.Series(flatten(row_c), index=row.index)

In [ ]:
# apply the function to every row
centered_df = df.apply(center_row, axis=1)
centered_df

## Sort by angle

In [ ]:
# given a list of points [(x,y), (x,y), ...]
# return list sorted by their angle to the origin.

def sort_by_angle(points):

  def angle(xy):
    x,y=xy
    return np.atan2(y, x) + np.pi # adding pi so angle is between [0,2pi] and not [-pi,pi]

  return sorted(points, key=angle)

In [ ]:
# TODO: got through each row,
#       turn vals into points (unflatten),
#       sort points by angle,
#       turn points into vals (flatten)

In [ ]:
# visualize centered or centered+sorted points

dfidx = 0

xs = centered_sorted_df.iloc[dfidx, 0::2]
ys = centered_sorted_df.iloc[dfidx, 1::2]

plt.axis("equal")
plt.plot(xs, ys, linestyle="-", marker="o", markersize=2)

x08 = centered_sorted_df.iloc[dfidx, 0:8:2]
y08 = centered_sorted_df.iloc[dfidx, 1:8:2]

plt.plot(x08, y08, linestyle="", marker="o", markersize=4)

plt.show()